In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv("visa.csv",sep=";", index_col="UserID")
df.head()

,FullName,Age,Occupation,AnnualIncome_TRY,TravelHistoryScore,TargetCountry,IsSchengen,VisaType,Status,IsSuccessful,RejectionReason
UserID,,,,,,,,,,,
1,Arş. Gör. Apaydın İnönü,23,Student,0,2,Spain,1,Family/Friend Visit,Rejected,0,Article 8: Invalid Travel Insurance
2,Uyanser Hançer,50,Teacher,512325,5,Italy,1,Business,Rejected,0,Article 3: Insufficient Funds
3,Yrd. Doç. Şevketfeza Özyurt Sezgin,24,Manager,1804976,9,Italy,1,Business,Approved,1,NaN
4,Bay Altınkaya Hürdoğan Güçlü,22,Freelancer,0,0,Belgium,1,Tourist,Rejected,0,Article 3: Insufficient Funds
5,Sanur Demir Akçay,50,Engineer,464905,6,Spain,1,Family/Friend Visit,Approved,1,NaN


In [20]:
df["IsSchengen"].value_counts()

IsSchengen
1    5000
Name: count, dtype: int64

In [21]:
df["Occupation"].value_counts()

Occupation
Gov_Officer    1009
Engineer        902
Freelancer      726
Teacher         668
Unemployed      543
Student         458
Manager         439
Doctor          255
Name: count, dtype: int64

In [22]:
df["TargetCountry"].value_counts()

TargetCountry
Spain          678
Belgium        658
Netherlands    635
Greece         618
Italy          609
France         603
Germany        602
Austria        597
Name: count, dtype: int64

In [23]:
df["TargetCountry"].value_counts()

TargetCountry
Spain          678
Belgium        658
Netherlands    635
Greece         618
Italy          609
France         603
Germany        602
Austria        597
Name: count, dtype: int64

In [24]:
df["VisaType"].value_counts()

VisaType
Tourist                1260
Business               1258
Family/Friend Visit    1241
Student                1241
Name: count, dtype: int64

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 1 to 5000
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   FullName            5000 non-null   object
 1   Age                 5000 non-null   int64 
 2   Occupation          5000 non-null   object
 3   AnnualIncome_TRY    5000 non-null   int64 
 4   TravelHistoryScore  5000 non-null   int64 
 5   TargetCountry       5000 non-null   object
 6   IsSchengen          5000 non-null   int64 
 7   VisaType            5000 non-null   object
 8   Status              5000 non-null   object
 9   IsSuccessful        5000 non-null   int64 
 10  RejectionReason     1765 non-null   object
dtypes: int64(5), object(6)
memory usage: 468.8+ KB


# DATA PREPROCESSING

In [26]:
df = df.drop(["FullName","Status", "RejectionReason","IsSchengen"], axis=1)

In [27]:
df.head()

,Age,Occupation,AnnualIncome_TRY,TravelHistoryScore,TargetCountry,VisaType,IsSuccessful
UserID,,,,,,,
1,23,Student,0,2,Spain,Family/Friend Visit,0
2,50,Teacher,512325,5,Italy,Business,0
3,24,Manager,1804976,9,Italy,Business,1
4,22,Freelancer,0,0,Belgium,Tourist,0
5,50,Engineer,464905,6,Spain,Family/Friend Visit,1


# Train Test Split

In [28]:
from sklearn.model_selection import train_test_split
X=df.drop("IsSuccessful", axis=1)
y= df["IsSuccessful"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=42)

In [30]:

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

cat_cols = ['Occupation', 'TargetCountry', 'VisaType']
num_cols = ['Age', 'AnnualIncome_TRY', 'TravelHistoryScore']

# remainder = passthrough yazınca garip bir şekilde hata alıyoruz. O hata bu şekilde düzeltildi.
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols),
        ('num', FunctionTransformer(), num_cols)
    ]
)
X_train_final = preprocessor.fit_transform(X_train)
X_test_final = preprocessor.transform(X_test)



# DATA NORMALIZATION

In [31]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

X_train_final = ss.fit_transform(X_train_final)
X_test_final = ss.transform(X_test_final)

# LAZY REGRESSOR

In [32]:
# Classifier algoritması seçmek için lazy regressor yapıyoruz
from lazypredict.Supervised import LazyClassifier
reg = LazyClassifier(verbose=1, ignore_warnings=True, custom_metric=None)
models, predictions = reg.fit(X_train_final, X_test_final, y_train, y_test)

print(models)

  0%|          | 0/29 [00:00<?, ?it/s]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.9206666666666666, 'Balanced Accuracy': 0.9149948559670782, 'ROC AUC': 0.9149948559670782, 'F1 Score': 0.9208173818102411, 'Time taken': 0.7500414848327637}
{'Model': 'BaggingClassifier', 'Accuracy': 0.9146666666666666, 'Balanced Accuracy': 0.9142583239805462, 'ROC AUC': 0.9142583239805462, 'F1 Score': 0.9152682286267191, 'Time taken': 0.4550042152404785}
{'Model': 'BernoulliNB', 'Accuracy': 0.902, 'Balanced Accuracy': 0.8945356341189674, 'ROC AUC': 0.8945356341189675, 'F1 Score': 0.9021861775302981, 'Time taken': 0.08393621444702148}
{'Model': 'CalibratedClassifierCV', 'Accuracy': 0.922, 'Balanced Accuracy': 0.9173213617658063, 'ROC AUC': 0.9173213617658061, 'F1 Score': 0.9222117167634174, 'Time taken': 0.5100061893463135}
{'Model': 'DecisionTreeClassifier', 'Accuracy': 0.8806666666666667, 'Balanced Accuracy': 0.8711536662925552, 'ROC AUC': 0.8711536662925553, 'F1 Score': 0.8808933726389344, 'Time taken': 0.13818907737731934}
{'Model': 'Dum

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=1000)

param_grid = [
    {
        'penalty': ['l1', 'l2'], 
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['liblinear'] # l1 için uygun solver
    },
    {
        'penalty': ['l2'], 
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['lbfgs', 'newton-cg'] # Genel solverlar
    }
]
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_final, y_train)

print({grid_search.best_params_})
print({grid_search.best_score_})


En iyi Parametreler: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
En iyi Skor: 0.924


# MODEL SAVING

In [34]:
import pickle

In [35]:
# Modeli kaydedip pyCharma atıyoruz.
with open("visa_load.pkl", "wb") as model_file:
    pickle.dump({
        "model": grid_search.best_estimator_,
        "encoders": preprocessor,
        "scaler": ss
    }, model_file)